# CRM using Simpy

CRM system simulation using simpy. 

Company CRM populated with accounts (customer companies) and opportunities coming from time to time from these accounts and which can be bidded, won or lost by Company.

Principles: Account "agents" wich will go through a set of events, namely:
- inbound_mktg_event: account finds the contant and follows a CTA to be added into Company MQL list
- outbount_mkt_event: account reached by Company outbound marketing actions and follows a CTA to be added into Company MQL
- inbound_sales_event: account is identified as a sales qualified leads and converted into prospect
- outbound_sales_event: account reached out by Company sales representative and accepts call to become a prospect
and other similar events representing how an account moves from stage to stage through the sales funnel and sales process

Each event is the result of:
- the current phase where the account is 
- an adjustable random variable, using some paramaters (example, conversion rate for CTA, ...)

In [1]:
import random
import simpy

from eccore.ipython import nb_setup
nb_setup()

from agents import Account, account_arrival, accounts_created_before, periodic_reporter
from agents import LEAD_CONVERSION_RATES, DELAY_RANGES

from enums import AccountStage, AccountType

Added path: /home/vtec/projects/diyai/crm-sim/src
Set autoreload mode


In [3]:
env.timeout?

Signature: env.timeout(delay: 'SimTime', value: 'Optional[Any]' = None)
Docstring:
A :class:`~simpy.events.Event` that gets processed after a *delay* has
passed.

This event is automatically triggered when it is created.
Type:      method

In [2]:
random.seed(1988)
env = simpy.Environment()
env.accounts = []
Account.cnt = 0
initial_accounts = 5
arrival_rate = 0.001  # on average, one new account every 2 units of time
env.process(account_arrival(env, initial_accounts, arrival_rate))
env.process(periodic_reporter(env, 2))
env.run(until=50)

[0.00] Acct 1: Account created: Acct 1 (UID: 797aa28e-28b7-4417-aa59-f0d202b09f6f) at stage LEAD
[0.00] Acct 2: Account created: Acct 2 (UID: 497f2c73-b8ba-4d74-8310-90279e44b3d2) at stage LEAD
[0.00] Acct 3: Account created: Acct 3 (UID: 5521631c-f92d-40a1-8c8b-06abad86286e) at stage LEAD
[0.00] Acct 4: Account created: Acct 4 (UID: a02b9108-7cd0-4c2e-b0cd-e2d5c9d48245) at stage LEAD
[0.00] Acct 5: Account created: Acct 5 (UID: aa133f8e-256f-4f05-a74a-f3e2210e51cc) at stage LEAD
[0.00] Acct 1: entering run for Acct 1, LEAD
[0.00] Acct 1: Lead is converting to MQL via inbound marketing event
[0.00] Acct 1: entering run for Acct 1, LEAD
[0.00] Acct 1: Lead is converting to MQL via inbound marketing event
[0.00] Acct 2: entering run for Acct 2, LEAD
[0.00] Acct 2: Lead is converting to SQL via outbound sales event
[0.00] Acct 2: entering run for Acct 2, LEAD
[0.00] Acct 2: Lead is converting to MQL via inbound marketing event
[0.00] Acct 3: entering run for Acct 3, LEAD
[0.00] Acct 3: Le

In [17]:
[a.name for a in env.accounts]

['Acct 1', 'Acct 2', 'Acct 3']

In [30]:
accounts_created_before(env=env, t=18)

# Sandbox

## Concept of store

In [16]:
class Account:
    def __init__(self, env, name):
        self.env = env
        self.name = name
        self.inbox = simpy.Store(env)
        self.env.accounts.append(self)
        print(f"Created {self.name}")

    def run(self):
        while True:
            msg = yield self.inbox.get()
            print(f"{self.env.now}: Account {self.name} received {msg}")

def report(env):
    print(f"\n--- Report at time {env.now} ---")
    for account in env.accounts:
        print(f"{account.name}: {account.inbox.items}")
    print("-------------------------------\n")

# ... Account class and other functions ...

def periodic_reporter(env, interval):
    # First report at time 1
    yield env.timeout(1)
    report(env)
    while True:
        yield env.timeout(interval)
        report(env)

In [17]:
def agent(env, account_store):
    yield env.timeout(5)
    yield account_store.put("marketing outreach")
    print(f"{env.now}: Agent sent event to account")

In [20]:
env = simpy.Environment()
env.accounts = []
account1 = Account(env, "A1")
account2 = Account(env, "A2")

env.process(account1.run())
env.process(account2.run())

env.process(periodic_reporter(env, 1))

Created A1
Created A2


<Process(periodic_reporter) object at 0x7900ed1fd870>

In [21]:
env.process(agent(env, account1.inbox))
env.run(until=10)



--- Report at time 1 ---
A1: []
A2: []
-------------------------------


--- Report at time 2 ---
A1: []
A2: []
-------------------------------


--- Report at time 3 ---
A1: []
A2: []
-------------------------------


--- Report at time 4 ---
A1: []
A2: []
-------------------------------


--- Report at time 5 ---
A1: ['marketing outreach']
A2: []
-------------------------------

5: Agent sent event to account
5: Account A1 received marketing outreach

--- Report at time 6 ---
A1: []
A2: []
-------------------------------


--- Report at time 7 ---
A1: []
A2: []
-------------------------------


--- Report at time 8 ---
A1: []
A2: []
-------------------------------


--- Report at time 9 ---
A1: []
A2: []
-------------------------------



In [15]:
account1.inbox.put("new message")
account1.inbox.put("another message")

account1.inbox.items

['new message', 'another message']